## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,35.47,84,100,2.44,overcast clouds
1,1,Hobart,AU,-42.8794,147.3294,48.06,70,40,1.99,scattered clouds
2,2,Auki,SB,-8.7676,160.7034,74.16,87,5,4.43,clear sky
3,3,Sambava,MG,-14.2667,50.1667,69.66,80,0,12.01,clear sky
4,4,Namwala,ZM,-15.7504,26.4384,65.32,30,0,6.35,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 50


In [4]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                          
preferred_cities_df.head(10) 

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,35.47,84,100,2.44,overcast clouds
1,1,Hobart,AU,-42.8794,147.3294,48.06,70,40,1.99,scattered clouds
6,6,New Norfolk,AU,-42.7826,147.0587,47.71,68,99,1.01,overcast clouds
7,7,Busselton,AU,-33.6500,115.3333,48.04,90,1,6.22,clear sky
19,19,Ilulissat,GL,69.2167,-51.1000,48.22,76,1,4.61,clear sky
33,33,Kaitangata,NZ,-46.2817,169.8464,40.95,87,100,10.09,overcast clouds
45,45,Barrow,US,71.2906,-156.7887,44.62,93,40,8.05,scattered clouds
46,46,Nanortalik,GL,60.1432,-45.2371,45.91,93,100,4.32,overcast clouds
48,48,Bluff,NZ,-46.6000,168.3333,35.80,76,100,3.47,overcast clouds
50,50,Arman,RU,59.7000,150.1667,45.21,64,63,10.42,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
np.where(pd.isnull(preferred_cities_df))

(array([], dtype=int64), array([], dtype=int64))

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()


In [7]:
preferred_cities_df.count()


Unnamed: 0             54
City                   54
Country                54
Lat                    54
Lng                    54
Max Temp               54
Humidity               54
Cloudiness             54
Wind Speed             54
Current Description    54
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,35.47,overcast clouds,-46.1927,168.8643,
1,Hobart,AU,48.06,scattered clouds,-42.8794,147.3294,
6,New Norfolk,AU,47.71,overcast clouds,-42.7826,147.0587,
7,Busselton,AU,48.04,clear sky,-33.6500,115.3333,
19,Ilulissat,GL,48.22,clear sky,69.2167,-51.1000,
33,Kaitangata,NZ,40.95,overcast clouds,-46.2817,169.8464,
45,Barrow,US,44.62,scattered clouds,71.2906,-156.7887,
46,Nanortalik,GL,45.91,overcast clouds,60.1432,-45.2371,
48,Bluff,NZ,35.80,overcast clouds,-46.6000,168.3333,
50,Arman,RU,45.21,broken clouds,59.7000,150.1667,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {'radius': 5000, 'type': 'lodging', 'key': g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,35.47,overcast clouds,-46.1927,168.8643,Ellie's Villa
1,Hobart,AU,48.06,scattered clouds,-42.8794,147.3294,Mantra on Collins Hobart
6,New Norfolk,AU,47.71,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
7,Busselton,AU,48.04,clear sky,-33.6500,115.3333,Observatory Guest House
19,Ilulissat,GL,48.22,clear sky,69.2167,-51.1000,Hotel Arctic
33,Kaitangata,NZ,40.95,overcast clouds,-46.2817,169.8464,Kaitangata Motor Camp
45,Barrow,US,44.62,scattered clouds,71.2906,-156.7887,King Eider Inn
46,Nanortalik,GL,45.91,overcast clouds,60.1432,-45.2371,Hotel Kap Farvel
48,Bluff,NZ,35.80,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
50,Arman,RU,45.21,broken clouds,59.7000,150.1667,


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,35.47,overcast clouds,-46.1927,168.8643,Ellie's Villa
1,Hobart,AU,48.06,scattered clouds,-42.8794,147.3294,Mantra on Collins Hobart
6,New Norfolk,AU,47.71,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
7,Busselton,AU,48.04,clear sky,-33.6500,115.3333,Observatory Guest House
19,Ilulissat,GL,48.22,clear sky,69.2167,-51.1000,Hotel Arctic
33,Kaitangata,NZ,40.95,overcast clouds,-46.2817,169.8464,Kaitangata Motor Camp
45,Barrow,US,44.62,scattered clouds,71.2906,-156.7887,King Eider Inn
46,Nanortalik,GL,45.91,overcast clouds,60.1432,-45.2371,Hotel Kap Farvel
48,Bluff,NZ,35.80,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
50,Arman,RU,45.21,broken clouds,59.7000,150.1667,


In [12]:
# 8a. Create the output File (CSV)
pd.DataFrame.to_csv(hotel_df, 'WeatherPy_vacation.csv',)

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} at {Max Temp}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))